# Neural network tuned with BOHB

## Importing

In [ ]:
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres
from hpbandster.optimizers import BOHB
import hpbandster.visualization as hpvis
import logging
import pickle
logging.getLogger('hpbandster').setLevel(logging.DEBUG)

sys.path.append("../")
from func.load_data import prepare_dataloaders, load_data_from_file

## Load data

In [ ]:
X_train, X_metafeatures_train, y_train, X_val, X_metafeatures_val, y_val, X_test, X_metafeatures_test, y_test    =   load_data_from_file("cached/six_datasets_lw.json", "cached/metafeatures_6.json")

## Prepare data

In [ ]:
batch_size = 4

train_dataloader = prepare_dataloaders(X_hp=X_train, X_mf=X_metafeatures_train, y= y_train, scaling="minmax",batch_size=batch_size, typeD = "tensor")
validation_dataloader = prepare_dataloaders(X_hp=X_val, X_mf=X_metafeatures_val, y= y_val, scaling="minmax",batch_size=batch_size, typeD = "tensor")
test_dataloader = prepare_dataloaders(X_hp=X_test, X_mf=X_metafeatures_test, y= y_test, scaling="minmax",batch_size=batch_size, 
typeD = "tensor")

In [ ]:
for x,  y in train_dataloader:
    print("X- minibatched: ", x)
    print("y- minibatched: ", y)
    break

## Sanity test for out worker

In [ ]:
working_dir = os.curdir
# minimum budget that BOHB uses
min_budget = 1
# largest budget BOHB will use
max_budget = 9
worker = PyTorchWorker(run_id='0', input_size=55, output_size=1, train_loader= train_dataloader, validation_loader= validation_dataloader, test_loader= test_dataloader)
cs = worker.get_configspace()
config = cs.sample_configuration().get_dictionary()
print("Config: ", config)

In [ ]:
res = worker.compute(config=config, budget=min_budget, working_directory=working_dir)
print(res)

## BOHB parameters and variables

In [ ]:
result_file = os.path.join(working_dir, 'bohb_LC_result.pkl')
nic_name = 'lo0'
port = 0
run_id = 'bohb_run_1'
n_bohb_iterations = 12

In [ ]:
try:
    try:
        host = hpns.nic_name_to_host(nic_name)
    except ValueError as e:
        host = "localhost"
        print(e)
        print("ValueError getting host from nic_name {}, "
              "setting to localhost.".format(nic_name))    
    
    ns = hpns.NameServer(run_id=run_id, host=host, port=port,
                         working_directory=working_dir)
    ns_host, ns_port = ns.start()

    # Start local worker
    w = PyTorchWorker(run_id=run_id, host=host, nameserver=ns_host,
                      nameserver_port=ns_port, timeout=120, input_size=55, output_size=1, train_loader= train_dataloader, validation_loader= validation_dataloader, test_loader= test_dataloader)
    w.run(background=True)

    # Run an optimizer
    bohb = BOHB(configspace=worker.get_configspace(),
                run_id=run_id,
                host=host,
                nameserver=ns_host,
                nameserver_port=ns_port,
                min_budget=min_budget, max_budget=max_budget)

    result = bohb.run(n_iterations=n_bohb_iterations)
    print("Write result to file {}".format(result_file))
    with open(result_file, 'wb') as f:
        pickle.dump(result, f)
finally:
    bohb.shutdown(shutdown_workers=True)
    ns.shutdown()

## Get best run

In [ ]:
inc_id = result.get_incumbent_id()  # get config_id of incumbent (lowest loss)
inc_run = result.get_runs_by_id(inc_id)[-1]  # get run with this config_id on highest budget
best_error, best_model = inc_run.loss, inc_run.info['model']
print("The best model (config_id {}) has the lowest final error with {:.4f}."
      .format(inc_run.config_id, best_error))
print(best_model)

In [ ]:
all_runs = result.get_all_runs()
id2conf = result.get_id2config_mapping()

In [ ]:
hpvis.finished_runs_over_time(all_runs, show=True)

In [ ]:
hpvis.correlation_across_budgets(result, show=True)

In [ ]:
hpvis.losses_over_time(all_runs, show=True)

In [ ]:
hpvis.performance_histogram_model_vs_random(all_runs, id2conf, show=True)